In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from math import exp
import matplotlib.gridspec as gridspec
import matplotlib.ticker as mtick
from matplotlib.lines import Line2D

In [ ]:
df_wave1 = pd.read_csv('meta_regression_result_wave1.csv')
df_wave2 = pd.read_csv('meta_regression_result_wave2.csv')
df_wave3 = pd.read_csv('meta_regression_result_wave3.csv')
df_wave4 = pd.read_csv('meta_regression_result_wave4.csv')
df = pd.DataFrame()
for n, _df in enumerate([df_wave1, df_wave2, df_wave3, df_wave4]):
    _df['cov'] = ['intrcpt', 'age_group30-69', 'age_group70-90', 'SexMale', 'IncomeQ1', 'IncomeQ2', 'IncomeQ3', 'IncomeQ4',
                       'EducationLow', 'Office> 50%', 'OfficeOther']
    _df['effect'] = _df['pred'] -1
    _df['effect_ci_lb'] = _df['ci.lb'] -1
    _df['effect_ci_ub'] = _df['ci.ub'] -1
    _df = _df.set_index('cov')
    _df.columns = [i+'_wave'+str(n+1) for i in _df.columns]
    if len(df) == 0:
        df = _df
    else:
        df = pd.concat([df, _df], axis = 1)
df

In [ ]:
df = df.rename(index = {'intrcpt': 'Ref.', 'age_group30-69': 'Age: 30-69', 'age_group70-90': 'Age: 70-90',
                        'SexMale': 'Sex: male', 'IncomeQ1': 'Income: Q1', 'IncomeQ2': 'Income: Q2', 'IncomeQ3': 'Income: Q3',
                        'IncomeQ4': 'Income: Q4', 'EducationLow': 'Education: low', 'Office> 50%': 'Remote work: no', 'OfficeOther': 'Remote work: other'})
df

In [ ]:
df_wave1_mean_difference = pd.read_csv('meta_regression_result_wave1_mean_difference.csv')
df_wave2_mean_difference = pd.read_csv('meta_regression_result_wave2_mean_difference.csv')
df_wave3_mean_difference = pd.read_csv('meta_regression_result_wave3_mean_difference.csv')
df_wave4_mean_difference = pd.read_csv('meta_regression_result_wave4_mean_difference.csv')
df_mean_difference = pd.DataFrame()
for n, _df in enumerate([df_wave1_mean_difference, df_wave2_mean_difference, df_wave3_mean_difference, df_wave4_mean_difference]):
    _df['cov'] = ['intrcpt', 'age_group30-69', 'age_group70-90', 'SexMale', 'IncomeQ1', 'IncomeQ2', 'IncomeQ3', 'IncomeQ4',
                       'EducationLow', 'Office> 50%', 'OfficeOther']
    _df = _df.set_index('cov')
    _df.columns = [i+'_wave'+str(n+1) for i in _df.columns]
    if len(df) == 0:
        df_mean_difference = _df
    else:
        df_mean_difference = pd.concat([df_mean_difference, _df], axis = 1)
df_mean_difference

In [ ]:
def change_colors(row, wave):
    if row.name == 'intrcpt':
        return 'black'
    else:
        if (1< row['ci.ub_%s'%wave]) and  ((1> row['ci.lb_%s'%wave])):
            return 'gray'
        else:
            return 'purple'

In [ ]:
wave = 'wave1'
df_mean_difference.apply(change_colors, wave = wave, axis = 1)

In [ ]:
fig, axes = plt.subplots(2,2,figsize = (12,10), dpi = 350)
axes = axes.flat
for i, wave in enumerate(['wave1', 'wave2', 'wave3', 'wave4']):
    ax = axes[i]
    colors = df_mean_difference.apply(change_colors, wave = wave, axis = 1)
    ax.barh(range(11), df['effect_%s'%wave], align = 'center', color = colors,alpha = 0.5)
    ax.set_yticks(range(11))
    ax.set_yticklabels(df.index, fontsize = 15)
    for ticklabel, tickcolor in zip(ax.get_yticklabels(), colors):
        ticklabel.set_color(tickcolor)
    ax.invert_yaxis()
    ax.spines[['right', 'top']].set_visible(False)
    ax.axvline(0, color = 'gray', linewidth = 1.5, linestyle = '--')
    ax.set_title(wave, fontsize = 18)
    ax.set_xlim([-0.25, 0.8])
    ax.set_xlabel('Percentage change (in daily steps)', fontsize = 13)
    
    ax2 = ax.twiny()
    ax2.set_xlim([0.2, 1.5])
    ax2.set_xlabel('RRR', fontsize = 13)
    error_minus = df_mean_difference['pred_%s'%wave] - df_mean_difference['ci.lb_%s'%wave]
    error_plus = df_mean_difference['ci.ub_%s'%wave] - df_mean_difference['pred_%s'%wave]
    j = 0
    for index, row in df_mean_difference.iterrows():
        ax2.errorbar(x = row['pred_%s'%wave],y = j, xerr = np.array(error_minus[j], error_plus[j]).T, marker = 'o',ls = 'none', capsize = 5, color = colors[j], markeredgewidth=1.5, linewidth = 1.5, alpha = 0.5)
        j+=1
    ax2.axvline(1, color = 'gray', linewidth = 1.5, alpha = 0.5)
fig.tight_layout()
#plt.savefig('Meta_regression_visualization.pdf')

In [ ]:
fig, axes = plt.subplots(1,2,figsize = (12,5), dpi = 300)
for i, wave in enumerate(['wave1', 'wave3']):
    colors = df_mean_difference.apply(change_colors, wave = wave, axis = 1)
    axes[i].barh(df['effect_%s'%wave].index, width = df['effect_%s'%wave], color = colors, alpha = 0.5)
    axes[i].set_xlim([-0.25, 0.25])
    axes[i].spines[['right', 'top']].set_visible(False)
    axes[i].axvline(0, color = 'gray', linewidth = 1.5, linestyle = '--')
    axes[i].invert_yaxis()
    axes[i].set_title(wave, fontsize = 18)
    axes[i].set_yticklabels(df.index, fontsize = 15)
    axes[i].set_xlabel('Percentage change (in daily steps)', fontsize = 13)
    vals = axes[i].get_xticks()
    axes[i].set_xticklabels(['{:,.0%}'.format(x) for x in vals])
    for ticklabel, tickcolor in zip(axes[i].get_yticklabels(), colors):
            ticklabel.set_color(tickcolor)
plt.tight_layout()
#plt.savefig('monday_meeting.svg')

In [ ]:
fig, axes = plt.subplots(2,2,figsize = (12,10), dpi = 350)
axes = axes.flat
for i, wave in enumerate(['wave1', 'wave2', 'wave3', 'wave4']):
    ax = axes[i]
    error_minus = df['pred_%s'%wave] - df['ci.lb_%s'%wave]
    error_plus = df['ci.ub_%s'%wave] - df['pred_%s'%wave]
    ax.errorbar(x = df['pred_%s'%wave],y = range(11), xerr = (error_minus, error_plus), marker = 'o',ls = 'none', capsize = 5, c = 'black', markeredgewidth=1.5, linewidth = 1.5)
    ax.set_xlim(0.6, 1.4)
    ax.set_yticks(range(11))
    ax.set_yticklabels(df.index, fontsize = 13)
    ax.invert_yaxis()
    ax.spines[['right', 'top']].set_visible(False)
    ax.axvline(1, color = 'gray', linestyle = (0,(1,10)), linewidth = 1.5)
    ax.set_title(wave, fontsize = 15)
fig.tight_layout()

In [ ]:
fig = plt.figure(figsize = (13,10), dpi = 350)
outer = gridspec.GridSpec(2, 2)
for i, wave in enumerate(['wave1', 'wave2', 'wave3', 'wave4']):
    inner = gridspec.GridSpecFromSubplotSpec(1, 2,
                    subplot_spec=outer[i], wspace = 0.13)
    ax1 = plt.Subplot(fig, inner[0])
    colors = df_mean_difference.apply(change_colors, wave = wave, axis = 1)
    bars = ax1.barh(range(11), df['effect_%s'%wave], align = 'center', color = colors,alpha = 0.5)
    ax1.set_yticks(range(11))
    ax1.set_yticklabels(df.index, fontsize = 15)
    for ticklabel, tickcolor in zip(ax1.get_yticklabels(), colors):
        ticklabel.set_color(tickcolor)
    ax1.invert_yaxis()
    ax1.spines[['right']].set_visible(False)
    ax1.axvline(0, color = 'gray', linewidth = 1.5, linestyle = '--')
    ax1.set_xlim([-0.25, 0.2])
    ax1.set_xlabel('Percentage change\n(in daily steps)', fontsize = 10)
    fig.add_subplot(ax1)
    
    ax2 = plt.Subplot(fig, inner[1])
    ax2.set_xlim([0.8, 1.5])
    ax2.set_xlabel('RRR', fontsize = 10)
    error_minus = df_mean_difference['pred_%s'%wave] - df_mean_difference['ci.lb_%s'%wave]
    error_plus = df_mean_difference['ci.ub_%s'%wave] - df_mean_difference['pred_%s'%wave]
    j = 0
    for index, row in df_mean_difference.iterrows():
        ax2.errorbar(x = row['pred_%s'%wave],y = j, xerr = np.array(error_minus[j], error_plus[j]).T, marker = 'o',ls = 'none', capsize = 5, color = colors[j], markeredgewidth=1.5, linewidth = 1.5, alpha = 0.5)
        j+=1
    ax2.axvline(1, color = 'gray', linewidth = 1.5, alpha = 0.5)
    ax2.get_yaxis().set_visible(False)
    ax2.spines[['left']].set_visible(False)
    ax2.invert_yaxis()
    fig.add_subplot(ax2)
plt.figtext(0.33,0.998,"Wave1", va="center", ha="center", size=15)
plt.figtext(0.829,0.998,"Wave2", va="center", ha="center", size=15)
plt.figtext(0.33,0.505,"Wave3", va="center", ha="center", size=15)
plt.figtext(0.829,0.505,"Wave4", va="center", ha="center", size=15)
fig.tight_layout()
plt.savefig('Meta_regression_visualization.pdf', bbox_inches="tight")

# Visualization with GAMM (auto correlation and identity link)

In [ ]:
df_full = pd.read_csv(os.path.join('gamm meta regression results', 'gamm_identity_link_meta_regression_result_full.csv'))

In [ ]:
df_full = df_full.rename(columns = {'Unnamed: 0': 'name'})
df_full['wave'] = ['wave1']*11+['wave2']*11+['wave3']*11+['wave4']*11
df_full['name'] = df_full.apply(lambda row: row['name'] if row['wave'] == 'wave1' else row['name'][:-2], axis = 1)
df_full

In [ ]:
def assign_colors_full(row):
    if row['estimate'] != 0:
        if (0< row['upper']) and  ((0> row['lower'])):
            return 'gray'
        else:
            return 'purple'
    else:
        return 'gray'

In [ ]:
df_full['name'] = df_full['name'].replace({'intrcpt': 'Ref.', 'age_group30.69': 'Age: 30-69', 'age_group70.90': 'Age: 70-90',
                        'SexMale': 'Sex: male', 'IncomeQ1': 'Income: Q1', 'IncomeQ2': 'Income: Q2', 'IncomeQ3': 'Income: Q3',
                        'IncomeQ4': 'Income: Q4', 'EducationLow': 'Education: low', 'Office..50.': 'Remote work: no', 'OfficeOther': 'Remote work:\nunemployed'})
df_full

In [ ]:
df_full_ref = df_crude.drop(['lower', 'upper', 'estimate', 'pval'], axis = 1).rename(columns = {'index': 'name'})
def fill_data(row, column_name):
    query = df_full[(df_full['name'] == row['name']) & (df_full['wave'] == row['wave'])]
    if len(query) > 0:
        return query[column_name].values[0]
    else:
        return 0
df_full_ref['estimate'] = df_full_ref.apply(fill_data, column_name = 'estimate', axis = 1)
df_full_ref['lower'] = df_full_ref.apply(fill_data, column_name = 'lower', axis = 1)
df_full_ref['upper'] = df_full_ref.apply(fill_data, column_name = 'upper', axis = 1)
df_full_ref['name'] = df_full_ref['name'].replace({'Remote work: unemployed': 'Remote work:\nunemployed'})
df_full_ref

In [ ]:
fig, axes  = plt.subplots(2,2, figsize = (13,11), dpi = 350)
axes = axes.flat
for i, wave in enumerate(['wave1', 'wave2', 'wave3', 'wave4']):
    _df = df_full_ref[df_full_ref['wave'] == wave].reset_index(drop = True)
    _df.iloc[3], _df.iloc[4] = _df.iloc[4].copy(), _df.iloc[3].copy()
    print(_df)
    colors = list(_df.apply(assign_colors_full, axis = 1))
    error_plus = list(_df['upper'] - _df['estimate'])
    error_minus = list(_df['estimate'] - _df['lower'])
    j = 0
    for index, row in _df.iterrows():
        axes[i].errorbar(x = row['estimate'],y = j, xerr = np.array([error_minus[j], error_plus[j]]).reshape(2,1), marker = 'o',ls = 'none', capsize = 5, color = 'gray', markeredgewidth=1.5, linewidth = 1.5, alpha = 0.8)
        j+=1 
    axes[i].set_yticks(range(15))
    axes[i].set_yticklabels(_df['name'], fontsize = 15)
    for ticklabel, tickcolor in zip(axes[i].get_yticklabels(), colors):
        ticklabel.set_color('gray')
    axes[i].set_xlim(-1900, 1900)
    axes[i].axvline(0, color = 'gray', linewidth = 1.5, linestyle = '--')
    axes[i].invert_yaxis()
    axes[i].text(1150, 0, wave, fontsize = 15)
    axes[i].set_xlabel('Difference compared with reference level (steps)')
#custom_lines = [Line2D([0], [0], marker='o', color='white', label='reference',
#                          markerfacecolor='gray', markersize=8, alpha = 0.7),
#    Line2D([0], [0], color='purple', lw=2, marker='o', label ='significant', alpha = 0.5),
#                Line2D([0], [0], color='gray', lw=2, marker = 'o', label = 'insignificant', alpha = 0.5)]
#fig.legend(handles = custom_lines, ncol = 3, bbox_to_anchor = (0.69,-0.001))
fig.tight_layout()
plt.savefig('Meta_regression_adjusted_visualization.pdf', bbox_inches="tight")
plt.savefig('Meta_regression_adjusted_visualization.svg', bbox_inches="tight")

## Crude model

In [ ]:
df_crude_2 = pd.read_csv(os.path.join('gamm meta regression results', 'gamm_identity_link_meta_regression_result_crude_2.csv'))

df_crude_3 = pd.read_csv(os.path.join('gamm meta regression results', 'gamm_identity_link_meta_regression_result_crude_3.csv'))

df_crude_4 = pd.read_csv(os.path.join('gamm meta regression results', 'gamm_identity_link_meta_regression_result_crude_4.csv'))

df_crude_5 = pd.read_csv(os.path.join('gamm meta regression results', 'gamm_identity_link_meta_regression_result_crude_5.csv'))

df_crude_6 = pd.read_csv(os.path.join('gamm meta regression results', 'gamm_identity_link_meta_regression_result_crude_6.csv'))
def remove_suffix(row):
    if row['wave'] != 'wave1':
        return row['index'][:-1]
    else:
        return row['index']
for _df in [df_crude_2,df_crude_3,df_crude_4,df_crude_5,df_crude_6]:
    n_variables_per_wave = int(len(_df)/4)
    _df['wave'] = pd.Series(['wave1']*n_variables_per_wave +['wave2']*n_variables_per_wave+
                           ['wave3']*n_variables_per_wave+['wave4']*n_variables_per_wave)
    _df.columns = ['index', 'estimate', 'lower', 'upper','pval', 'wave']
    _df['index'] = _df.apply(remove_suffix, axis = 1)
df_crude_2['index'] = df_crude_2['index'].str.replace(r'intrcpt.*','age_group<30', regex = True)
df_crude_3['index'] = df_crude_3['index'].str.replace(r'intrcpt.*','sexFemale', regex = True)
df_crude_4['index'] = df_crude_4['index'].str.replace(r'intrcpt.*','IncomeQ5', regex = True)
df_crude_5['index'] = df_crude_5['index'].str.replace(r'intrcpt.*', 'EducationHigh', regex = True)
df_crude_6['index'] = df_crude_6['index'].str.replace(r'intrcpt.*',' Office< 50%', regex = True)
df_crude = pd.concat([df_crude_2,df_crude_3,df_crude_4,df_crude_5,df_crude_6])
df_crude

In [ ]:
df_crude['index'] = df_crude['index'].replace({'age_group<30': 'Age: <30','age_group30-69': 'Age: 30-69',
                                               'age_group70-90': 'Age: 70-90', 'SexMale': 'Sex: male', 'sexFemale': 'Sex: female',
                                               'IncomeQ1': 'Income: Q1', 'IncomeQ2': 'Income: Q2', 'IncomeQ3': 'Income: Q3',
                                               'IncomeQ5': 'Income: Q5','IncomeQ4': 'Income: Q4',
                                               'EducationLow': 'Education: low','EducationHigh': 'Education: high',
                                               'Office> 50%': 'Remote work: no',' Office< 50%': 'Remote work: yes',
                                               'OfficeOther': 'Remote work: unemployed'})
df_crude

In [ ]:
intercept_list = ['Age: <30', 'Sex: Female', 'Income: Q5', 'Education: High', 'Remote work: yes']
def assign_colors(row):
    if row['pval'] <= 0.05:
        return 'purple'
    else:
        return 'gray'

In [ ]:
df_crude['name_short'] = df_crude['index'].str.extract('.+:\s{1}(.+)')
df_crude

In [ ]:
fig, axes  = plt.subplots(2,2, figsize = (13,11), dpi = 350)
axes = axes.flat
for i, wave in enumerate(['wave1', 'wave2', 'wave3', 'wave4']):
    _df = df_crude[df_crude['wave'] == wave].reset_index(drop = True)
    _df.iloc[3], _df.iloc[4] =  _df.iloc[4].copy(), _df.iloc[3].copy()
#    _df.loc[_df['index'] == 'intrcpt', ['estimate', 'lower', 'upper']] = 0
    colors = list(_df.apply(assign_colors, axis = 1))
    error_plus = list(_df['upper'] - _df['estimate'])
    error_minus = list(_df['estimate'] - _df['lower'])
    j = 0
    for index, row in _df.iterrows():
        axes[i].errorbar(x = row['estimate'],y = j, xerr = np.array([error_minus[j], error_plus[j]]).reshape(2,1), marker = 'o',ls = 'none', capsize = 5, color = 'gray', markeredgewidth=1.5, linewidth = 1.5, alpha = 0.9)
        j+=1 
    axes[i].set_yticks(range(15))
    axes[i].set_yticklabels(_df['name_short'], fontsize = 15)
    for ticklabel, tickcolor in zip(axes[i].get_yticklabels(), colors):
        ticklabel.set_color('gray')
    axes[i].set_xlim(-1900, 1900)
    axes[i].invert_yaxis()
#     for k in [2.5, 4.5, 9.5, 11.5]:
#         axes[i].axhline(k, color = 'gray', linewidth = 1.5, alpha = 0.5, linestyle = '-')
    axes[i].axvline(0, color = 'gray', linewidth = 1, linestyle = '--')
    axes[i].text(1150,0,wave,fontsize = 15)
    axes[i].set_xlabel('Difference compared with pre-pandemic levels (steps)')
    
    axes[i].set_yticks([0.01,3.01, 5.01, 10.01, 12.2], minor=True)
    axes[i].set_yticklabels(['Age', 'Sex', 'Income', 'Education', 'Remote\nwork'], minor=True, fontsize = 15, c = 'black')
    axes[i].tick_params(axis='y', which='minor', length=0, pad=50)
    for j in [2.5, 4.5, 9.5, 11.5]:
        axes[i].axhline(j, xmin = -0.3, c = 'black', linewidth = 0.6, clip_on = False)
#custom_lines = [Line2D([0], [0], color='purple', lw=2, marker='o', label ='significant', alpha = 0.5),
#                Line2D([0], [0], color='gray', lw=2, marker = 'o', label = 'insignificant', alpha = 0.5)]
#fig.legend(handles = custom_lines, ncol = 2, bbox_to_anchor = (0.63,-0.001))
fig.tight_layout()
plt.savefig('Meta_regression_crude_visualization.pdf', bbox_inches="tight")
plt.savefig('Meta_regression_crude_visualization.svg', bbox_inches="tight")